<a href="https://colab.research.google.com/github/elhaddad2/Tasks_NLP_using_HUGGINGFACE/blob/main/Tasks_NLP_using_Hugginface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Tasks with Transformers using HuggingFace**
- Sentiment analysis
- Text generation
- Name entity recognition (NER)
- Question answering
- Filling masked text
- Summarization
- Translation
- Feature extraction

# **Import packages**

In [3]:
!pip install transformers sentencepiece

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.7 MB/s 
     |████████████████████████████████| 636 kB 61.1 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# **DataSet**

In [4]:
array_of_texts = [      
            ["morocco is being the most visited country in africa", "English"],
            ["morocco will become an emerging country in the world","English"],
            ["المغرب اجمل بلد من حيث الطبيعة" , "Arabic"],
            ["المغرب له طاقات مبدعة", "Arabic"],
            ["le maroc est un beau pays touristique", "French"],
            ["le maroc est un pays attractif à visiter", "French"]
]

print(array_of_texts)

[['morocco is being the most visited country in africa', 'English'], ['morocco will become an emerging country in the world', 'English'], ['المغرب اجمل بلد من حيث الطبيعة', 'Arabic'], ['المغرب له طاقات مبدعة', 'Arabic'], ['le maroc est un beau pays touristique', 'French'], ['le maroc est un pays attractif à visiter', 'French']]


In [5]:
from transformers import pipeline,AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
model = AutoModel.from_pretrained("akhooli/gpt2-small-arabic")

Some weights of the model checkpoint at akhooli/gpt2-small-arabic were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
my_classifier_using_pipeline = pipeline('sentiment-analysis' ,  model='akhooli/xlm-r-large-arabic-sent')

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Preprocessing data**

In [7]:
import string
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation  #all punctuations 

def preprocessing_data(text , type):
    #remove punctuations
 
  
    if type == "Arabic":

   
      #remove longation and stopwords
      text = re.sub("[إأآا]", "ا", text)
      text = re.sub("ى", "ي", text)
      text = re.sub("ؤ", "ء", text)
      text = re.sub("ئ", "ء", text)
      text = re.sub("ة", "ه", text)
      text = re.sub("گ", "ك", text)

    
    elif type == "English":

      text = text.lower()
      text = ''.join([item for item in text if item not in punctuations ])
      
 
    return text
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Text classification (Sentiments Analysis) - English, French and Arabic languages**

In [8]:
for text,language in array_of_texts:
  res = my_classifier_using_pipeline(preprocessing_data(text , language))[0]
  res['label'] = "Positive" if res['label'] == 'LABEL_2' else ("Negative" if res['label'] == 'LABEL_1'  else "Mixed")
  print("'{}' is {} {} text with occuracy = {}%".format(text,res['label'],language,round(res['score']*100, 2)))

'morocco is being the most visited country in africa' is Positive English text with occuracy = 54.71%
'morocco will become an emerging country in the world' is Positive English text with occuracy = 48.86%
'المغرب اجمل بلد من حيث الطبيعة' is Positive Arabic text with occuracy = 68.24%
'المغرب له طاقات مبدعة' is Positive Arabic text with occuracy = 62.57%
'le maroc est un beau pays touristique' is Positive French text with occuracy = 58.74%
'le maroc est un pays attractif à visiter' is Positive French text with occuracy = 58.59%


# **Text generation (English, French and Arabic languages)**

In [9]:
from transformers import pipeline, set_seed
generator_english_text = pipeline('text-generation', model='gpt2')
generator_arabic_text = pipeline('text-generation', model='akhooli/gpt2-small-arabic')
generator_french_text = pipeline('text-generation', model='dbddv01/gpt2-french-small')
set_seed(42)

In [10]:
for text , lang in array_of_texts:
  if lang == "Arabic":
    print(generator_arabic_text(text , max_length=30, num_return_sequences=5))
  elif lang == "English":
    print(generator_english_text(text , max_length=30, num_return_sequences=5))
  else:
    print(generator_french_text(text , max_length=30, num_return_sequences=5))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'morocco is being the most visited country in africa, selling out several hundred stores an hour at the fastest speeds.\n\nAnd there is'}, {'generated_text': 'morocco is being the most visited country in africa and in Mexico and in North American.\n\nHorticulture by nature\n\nMany'}, {'generated_text': 'morocco is being the most visited country in africa by visitors to the U.S., and according to a new census, more people are'}, {'generated_text': 'morocco is being the most visited country in africa with more than 8,000 visitors and nearly half of that comes from North America, according'}, {'generated_text': 'morocco is being the most visited country in africa, according to the Global Index of Countries for Conservation of Nature in 2012.\n\nIn'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'morocco will become an emerging country in the world\'s biggest economy and also one of the biggest global producers of energy."\n\nGross national'}, {'generated_text': 'morocco will become an emerging country in the world economy after his country was taken over by Italy on August 25, 2014. At that time,'}, {'generated_text': 'morocco will become an emerging country in the world of social media marketing at a time when there are a plethora of social and social media channels that'}, {'generated_text': 'morocco will become an emerging country in the world of food safety."\n\nShe said it was "too early to judge whether Fajardo'}, {'generated_text': 'morocco will become an emerging country in the world of cannabis industry, the potential of such a move could see big investments made by foreign financial institutions'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'المغرب اجمل بلد من حيث الطبيعة الخلابة والجافة إضافة إلى ذلك كله من حيث وجود بعض الآثار التاريخية، مما يعني ان هناك مناطق واسعة من اراضي'}, {'generated_text': 'المغرب اجمل بلد من حيث الطبيعة، فهو الوحيد من حيث الطبيعة. إذا كان أي من تلك الكائنات الحية، أو على عكس الأنواع الأخرى، كال'}, {'generated_text': 'المغرب اجمل بلد من حيث الطبيعة الخلابة، بالإضافة للعديد من المعالم الجميلة كالمرمل، القصر الملكي، الجرن، السوق، المدينة، قصر'}, {'generated_text': 'المغرب اجمل بلد من حيث الطبيعة و الطبيعة . و بالنسبة لنهار ذلك النوع، فان هذا النوع هو عنصر رئيسي في المناخ و في الصيف،'}, {'generated_text': 'المغرب اجمل بلد من حيث الطبيعة الخلابة بسبب مناخ البحر المتوسط من الشمال، إلى الشرق. وهو مناخ متوسط ورطب، وثلوج المرتفعات المرتفعة'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'المغرب له طاقات مبدعة تزيد عن ألفة في الثانية. ولقد بقي على هذه الطاقة طوال السنوات العشرين الماضية. كما أن هناك جهود كبيرة للعمل على'}, {'generated_text': 'المغرب له طاقات مبدعة. وقد قام بعمل مشروع القانون البيئي الدولي الذي كان عبارة عن اتفاقية تتعلق بوضع قانون الهواء النظيف من حيث المبدأ وتوفير المياه في'}, {'generated_text': 'المغرب له طاقات مبدعة على مر العصور، حتى يكون مع التقدم في مجال التعليم. ولكن بعد انتهاء العصر الجليدي الأخير، بدأ السكان في الدخول إلى'}, {'generated_text': 'المغرب له طاقات مبدعة، وهذه هي الغاية. وقد كان ذلك في أواخر السبعينيات وأوائل السبعينات. كان يعيش في مدينة بغداد من أم لبنانية. تلقى'}, {'generated_text': 'المغرب له طاقات مبدعة تزيد عن طاقة الصوت، فبهو الصوت قد يزيد على ما يقارب 30.6 تيرا هرتز، كما أن قدرة الصوت'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "le maroc est un beau pays touristique qui se veut la nouvelle. C'est le seul endroit où les touristes viennent profiter des eaux chaudes et frais,"}, {'generated_text': "le maroc est un beau pays touristique important avec ses nombreux jardins, son zoo, son système d'alimentation électrique, son cimetière, son théâtre, ses"}, {'generated_text': "le maroc est un beau pays touristique, célèbre en raison de l'activité économique et culturelle de la région et les activités touristiques qui s'y sont"}, {'generated_text': "le maroc est un beau pays touristique et culturel dont la plus grande population est de la ville, d'Algérie dans son ensemble. Les premiers édifices"}, {'generated_text': 'le maroc est un beau pays touristique, avec un tourisme culturel et un riche patrimoine national. Les touristes viennent le plus souvent à la côte, pour des'}]
[{'generated_text': "le maroc est un pays attractif à visiter, comme on l'a fait longtemps. Le territoire d'Empire romain a 

# **Question answering**

In [11]:
from transformers import pipeline

##### English context
question = "What is the capital of the Morocco?"
context = r"Morocco, a North African country bordering the Atlantic Ocean and Mediterranean Sea, is distinguished by its Berber, Arabian and European cultural influences. Marrakesh’s medina, a mazelike medieval quarter, offers entertainment in its Djemaa el-Fna square and souks (marketplaces) selling ceramics, jewelry and metal lanterns. The capital Rabat’s Kasbah of the Udayas is a 12th-century royal fort overlooking the water."
# Generating an answer to the question in context
qa_english_sentence = pipeline("question-answering")
answer = qa_english_sentence(question=question, context=context)
# Print the answer
print(f"English Question: {question}")
print(f"English Answer: '{answer['answer']}' with accuracy {answer['score']}")

##### French context
qa_french_sentence = pipeline('question-answering', model='fmikaelian/camembert-base-squad', tokenizer='fmikaelian/camembert-base-squad')
question_fr = "Quelle est la capitale du Maroc ?"
context_fr = "Le Maroc est un pays du nord-ouest de l'Afrique. Sa longue côte donnant sur l'océan Atlantique se termine au cap Spartel, limite occidentale du détroit de Gibraltar et où débute le littoral méditerranéen. Au sud du Maroc se trouve le territoire contesté Sahara occidental, revendiqué et contrôlé en grande partie par le Maroc. À l'est et au sud-est, le Maroc est limitrophe de l'Algérie. À l'ouest-sud-ouest et à quelque distance de la côte atlantique se situent les îles Canaries tandis qu'à 672 km à l'ouest-nord-ouest du littoral marocain, on rencontre Madère. Au nord du détroit de Gibraltar se trouve l'Espagne. La capitale administrative est Rabat. Parmi les grandes villes remarquables on trouve Casablanca, Agadir, Fès, Marrakech, Meknès, Tanger, Oujda, Nador."
french_answer = qa_french_sentence(question =  question_fr , context = context_fr)
# Print the answer
print(f"French Question: {question_fr}")
print(f"French Answer: '{french_answer['answer']}' with accuracy {french_answer['score']}")


English Question: What is the capital of the Morocco?
English Answer: 'Rabat’s Kasbah of the Udayas' with accuracy 0.7165263891220093



French Question: Quelle est la capitale du Maroc ?
French Answer: ' Rabat.' with accuracy 0.9795011878013611


# **Filling masked text in English,French and Arabic languages**

In [12]:
from transformers import pipeline
english_fill_mask = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking')
french_fill_mask  = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base")
arabic_fill_mask = pipeline('fill-mask', model='CAMeL-Lab/bert-base-camelbert-ca')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at CAMeL-Lab/bert-base-camelbert-ca were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
print(english_fill_mask(" I look to my country and i found it very [MASK]."))
print(french_fill_mask(" j'ai trouvé le match <mask>."))
print(arabic_fill_mask("لكل جواد [MASK] ."))

[{'sequence': 'i look to my country and i found it very strange.', 'score': 0.14268207550048828, 'token': 4326, 'token_str': 'strange'}, {'sequence': 'i look to my country and i found it very beautiful.', 'score': 0.10535375773906708, 'token': 3376, 'token_str': 'beautiful'}, {'sequence': 'i look to my country and i found it very warm.', 'score': 0.03969131037592888, 'token': 4010, 'token_str': 'warm'}, {'sequence': 'i look to my country and i found it very cool.', 'score': 0.035899579524993896, 'token': 4658, 'token_str': 'cool'}, {'sequence': 'i look to my country and i found it very different.', 'score': 0.033366329967975616, 'token': 2367, 'token_str': 'different'}]
[{'sequence': "j'ai trouvé le match nul.", 'score': 0.21992997825145721, 'token': 3817, 'token_str': 'nul'}, {'sequence': "j'ai trouvé le match intéressant.", 'score': 0.10411430150270462, 'token': 1953, 'token_str': 'intéressant'}, {'sequence': "j'ai trouvé le match magnifique.", 'score': 0.022644251585006714, 'token':

# **Summarization**

In [24]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [25]:
article_summary = """Today, humans and machines generate data faster than it is humanly possible to absorb and interpret it to make complex decisions. Artificial intelligence is the basis of all computer learning and represents the future of complex decision making. For example, most humans can learn not to lose in a simple game of tic-tac-toe, when there are 255,168 possible actions, 46,080 of which lead to a draw. In contrast, checkers champions are rarer, since there are more than 500 x 1018 (500 trillion) possible moves. Computers are able to calculate these combinations and the best possible permutations very efficiently, in order to make the right decision. AI (with its logical evolution, machine learning) and deep learning represent the future of decision making.
AI is present in our daily lives. For example, it is used by financial institutions' fraud detection services, for predicting purchase intentions and in interactions with online customer services. Here are some examples: 
Fraud detection. In the financial industry, artificial intelligence is used in two ways. Applications that score credit applications use AI to assess consumer creditworthiness. More advanced AI engines are responsible for monitoring and detecting fraudulent credit card payments in real time.
Virtual Customer Service (VCS). Call centers use VCS to predict and respond to customer requests without human intervention. Voice recognition and a human dialogue simulator are the first point of interaction with customer service. More complex requests require human intervention.
When an Internet user opens a chat window on a web page (chatbot), the interlocutor is often a computer running a specialized form of AI. If the chatbot fails to interpret the question or solve the problem, a human agent takes over. These interpretation failures are sent to the machine learning system to improve future interactions of the AI application.
"""

context_fr = """On pense en général que le Maroc est peuplé d´Arabes et de Berbères. Il est vrai que ces deux groupes sont dominants, même s’ils ne sont pas seuls. Les Arabes représentent environ la moitié de la population. Issus des tribus conquérantes, ils sont essentiellement citadins. Les Berbères (premiers occupants du Maroc, leur nom vient du latin barbarus), à l´opposé, forment l´essentiel de la population de la montagne et du désert. On en distingue généralement quatre groupes, parlant quatre variantes de la langue berbère : les Rifains, le groupe du Moyen-Atlas, les Berbères du Haut-Atlas et les groupes nomades des provinces du Sud (essentiellement des Reguibat). Bien que parlant une langue proche du tamachek, les Reguibat ne sont pas des Touaregs. Pour un Occidental, la différence la plus visible entre eux est la selle ; la selle reguibat est en forme de coquille et son ornementation est plus sobre. Une anecdote sur les Berbères du Haut-Atlas, qui sont essentiellement des Chleuhs. Leur réputation de guerriers indomptables est proverbiale au point que leur nom est devenu, pendant la Première Guerre mondiale, le sobriquet donné aux soldats allemands. La population juive est encore importante et jouit d´une position économique forte. Il est vrai que le Maroc a toujours été très tolérant à l’égard les Juifs, qui n´ont jamais subi d´avanies et sont parfaitement intégrés. Enfin, il faut signaler deux populations minoritaires : les Occidentaux (plus de 100 000 personnes) et, dans le sud, des Haratines, descendant des esclaves des populations nomades.
"""

print("Summary:",summarizer(article_summary, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])
print("Summary:",summarizer(context_fr, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Summary:  Artificial intelligence is the basis of all computer learning and represents the future of complex decision making . For example, it is used by financial institutions' fraud detection services, for predicting purchase intentions and in interactions with online customer services .
Summary:  On pense en général that le Maroc est peuplé d´Arabes et de Berbères . The Maroc a toujours été très tolérant à l’égard les Juifs, who n´ont jamais subi d´avanies and sont intégrés .


# **Translation**

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_ar = "المغرب أصبح قدوة في محابة الارهاب"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


['Morocco has become a role model in the fight against terrorism.']

# **Feature extraction**

In [2]:
from transformers import pipeline, AutoTokenizer

feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base")
features = feature_extraction("i am Phd Student.")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
print(features[0])
print(len(features[0])) #6

[[-0.029922068119049072, 0.10239661484956741, 0.014355168677866459, -0.09604029357433319, 0.0781591460108757, -0.06703353673219681, -0.0334012433886528, 0.09124678373336792, 0.06621362268924713, -0.06330303102731705, -0.06947078555822372, 0.046519119292497635, 0.037825994193553925, -0.022006306797266006, 0.050583191215991974, -0.009641393087804317, -0.010351193137466908, 0.05151741951704025, -0.017088117077946663, -0.0005844096885994077, -0.04602883756160736, 0.0015778698725625873, -0.04295230284333229, 0.11367052048444748, -0.008789235725998878, 0.07588732242584229, 0.13061979413032532, 0.06271223723888397, -0.021765928715467453, 0.019717657938599586, 0.002317583654075861, -0.06651708483695984, 0.01881229691207409, 0.06478279829025269, -0.010121201165020466, 0.0722135454416275, 0.051603931933641434, 0.04691149666905403, -0.02374407649040222, 0.08596079051494598, -0.00959633570164442, 0.025571759790182114, 0.047893326729536057, -0.018285099416971207, -0.03492620959877968, -0.0044203731